In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

# Config

In [ ]:
test_split = 0.05
validation_split = 0.05

learning_rate = 1e-3
epochs = 5
batch_size = 128

input_size = 300
bottleneck_size = 200
model_version = 4.1

alpha = 0.5 # How much the teacher should influence the latent representation

experiment_name = f"autoencoder_{input_size}to{bottleneck_size}_v{model_version}"
dataset = f"data/embeddings/base/clipped.glove.6B.{input_size}d.txt"
teacher_dataset = f"data/embeddings/base/clipped.glove.6B.{bottleneck_size}d.txt"

In [ ]:
gpu = torch.cuda.is_available()
device = torch.device("cuda" if gpu else "cpu")

In [ ]:
print(gpu, device)

# Load & Prepare Embeddings for Training

In [ ]:
def load_embeddings(dataset):
    words = []
    vectors = []
    with open(dataset, "r", encoding='utf8') as fp:
        for line in fp:
            line = line.split()
            word = line[0]
            vector = np.asarray(line[1:], 'float32')
            words.append(word)
            vectors.append(vector)
    vectors = torch.as_tensor(vectors)
    return words, vectors

In [ ]:
input_words, input_vectors = load_embeddings(dataset)

In [ ]:
teacher_words, teacher_vectors = load_embeddings(teacher_dataset)

In [ ]:
test_split = int(test_split * len(input_words))
validation_split = int(validation_split* len(input_words))

In [ ]:
print(len(input_words), input_vectors.shape)
print(len(teacher_words), teacher_vectors.shape)
print(test_split, validation_split)

In [ ]:
input_train_vectors, input_test_vectors, input_train_words, input_test_words = train_test_split(
    input_vectors, input_words, test_size=test_split, random_state=seed
)

In [ ]:
input_train_vectors, input_validation_vectors, input_train_words, input_validation_words = train_test_split(
    input_train_vectors, input_train_words, test_size=test_split, random_state=seed
)

In [ ]:
teacher_train_vectors, teacher_test_vectors, teacher_train_words, teacher_test_words = train_test_split(
    teacher_vectors, teacher_words, test_size=test_split, random_state=seed
)

In [ ]:
teacher_train_vectors, teacher_validation_vectors, teacher_train_words, teacher_validation_words = train_test_split(
    teacher_train_vectors, teacher_train_words, test_size=validation_split, random_state=seed
)

In [ ]:
input_train_vectors.shape, input_validation_vectors.shape, input_test_vectors.shape

In [ ]:
teacher_train_vectors.shape, teacher_validation_vectors.shape, teacher_test_vectors.shape

In [ ]:
# Note: We don't actually use these words since the model doesn't care about them.
# We just compute them in case we want to check some particular word or something.
len(input_train_words), len(input_validation_words), len(input_test_words)

In [ ]:
train_dataset = torch.utils.data.TensorDataset(input_train_vectors, teacher_train_vectors)
validation_dataset = torch.utils.data.TensorDataset(input_validation_vectors, teacher_validation_vectors)
test_dataset = torch.utils.data.TensorDataset(input_test_vectors, teacher_test_vectors)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=gpu
)
validation_dataloader = torch.utils.data.DataLoader(
    validation_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=gpu
)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=gpu
)

# Model Time

In [ ]:
class AutoEncoderWithoutHiddenLayer(nn.Module):
    def __init__(self, input_size, bottleneck_size):
        super().__init__()
        self.encoder = nn.Linear(in_features=input_size, out_features=bottleneck_size)
        self.encoder_activation = nn.Tanh()
        
        self.decoder = nn.Linear(in_features=bottleneck_size, out_features=input_size)
        self.decoder_activation = nn.Tanh()
        
        self.decoder.weight = nn.Parameter(self.encoder.weight.transpose(0,1))
    
    def forward(self, features):
        latent_representation = self.encoder_activation(self.encoder(features))
        reconstructed_input = self.decoder_activation(self.decoder(latent_representation))
        return reconstructed_input, latent_representation
    
    def encode(self, features):
        return self.encoder_activation(self.encoder(features))

In [ ]:
model = AutoEncoderWithoutHiddenLayer(input_size, bottleneck_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

In [ ]:
model

In [ ]:
for epoch in range(epochs):
    train_loss = 0
    train_reconstruction_loss = 0
    train_teacher_loss = 0
    validation_loss = 0
    validation_reconstruction_loss = 0
    validation_teacher_loss = 0
    
    # Training Loop
    for iteration, batch in enumerate(tqdm(train_dataloader)):
        # Reset gradients back to zero for this iteration
        optimizer.zero_grad()
        
        # Get our inputs & our teacher representation
        input_batch, teacher_batch = batch
        
        # Move batch to device
        input_batch = input_batch.to(device)

        # Run our model & get outputs
        outputs, latent_representation = model(input_batch)

        # Calculate reconstruction loss
        reconstruction_loss = criterion(outputs, input_batch)
        teacher_loss = alpha * criterion(latent_representation, teacher_batch)
        batch_loss = reconstruction_loss + teacher_loss 
                  
        # Backprop
        batch_loss.backward()
        
        # Update our optimizer parameters
        optimizer.step()
        
        # Add the batch's loss to the total loss for the epoch
        train_loss += batch_loss.item()
        train_reconstruction_loss += reconstruction_loss.item()
        train_teacher_loss += teacher_loss.item()
        
    # Validation Loop
    with torch.no_grad():
        for iteration, batch in enumerate(tqdm(validation_dataloader)):
            # Get our inputs & our teacher representation
            input_batch, teacher_batch = batch
            
            # Move batch to device
            input_batch = input_batch.to(device)

            # Run our model & get outputs
            outputs, latent_representation = model(input_batch)

            # Calculate reconstruction loss
            reconstruction_loss = criterion(outputs, input_batch)
            teacher_loss = alpha * criterion(latent_representation, teacher_batch)
            batch_loss = reconstruction_loss + teacher_loss

            # Add the batch's loss to the total loss for the epoch
            validation_loss += batch_loss.item()
            validation_reconstruction_loss += reconstruction_loss.item()
            validation_teacher_loss += teacher_loss.item()
    
    # Compute the average losses for this epoch
    train_loss = train_loss / len(train_dataloader)
    train_reconstruction_loss = train_reconstruction_loss / len(train_dataloader)
    train_teacher_loss = train_teacher_loss / len(train_dataloader)
    
    validation_loss = validation_loss / len(validation_dataloader)
    validation_reconstruction_loss = validation_reconstruction_loss / len(validation_dataloader)
    validation_teacher_loss = validation_teacher_loss / len(validation_dataloader)
    
    # Print Metrics
    print(
        f"Epoch: {epoch+1}/{epochs}, \
        \nTrain Reconstruction Loss = {train_loss} = {train_reconstruction_loss} + {train_teacher_loss}, \
        \nValidation Reconstruction Loss = {validation_loss} = {validation_reconstruction_loss} + {validation_teacher_loss}"
    )

# Test Model

In [ ]:
model.eval()

In [ ]:
test_loss = 0
test_reconstruction_loss = 0
test_teacher_loss = 0

# Testing Loop
with torch.no_grad():
    for iteration, batch in enumerate(tqdm(test_dataloader)):
        # Get our inputs & our teacher representation
        input_batch, teacher_batch = batch

        # Move batch to device
        input_batch = input_batch.to(device)

        # Run our model & get outputs
        outputs, latent_representation = model(input_batch)

        # Calculate reconstruction loss
        reconstruction_loss = criterion(outputs, input_batch)
        teacher_loss = alpha * criterion(latent_representation, teacher_batch)
        batch_loss = reconstruction_loss + teacher_loss

        # Add the batch's loss to the total loss for the epoch
        test_loss += batch_loss.item()
        test_reconstruction_loss += reconstruction_loss.item()
        test_teacher_loss += teacher_loss.item()

# Compute the average losses for this epoch
test_loss = test_loss / len(test_dataloader)
test_reconstruction_loss = test_reconstruction_loss / len(test_dataloader)
test_teacher_loss = test_teacher_loss / len(test_dataloader)

# Print Metrics
print(
        f"Train Reconstruction Loss = {test_loss} = {test_reconstruction_loss} + {test_teacher_loss}"
    )

# Generate Latent Embeddings

In [ ]:
latent_vectors = {}
with torch.no_grad():
    for i, (word, vector) in enumerate(tqdm(zip(input_words, input_vectors))):
        latent_vectors[word] = model.encode(vector)

In [ ]:
len(latent_vectors)

In [ ]:
latent_vectors['the']

# Save Model & Embeddings

In [ ]:
torch.save(model.state_dict(), f"models/{experiment_name}.pt")

In [ ]:
# Need to convert the latent embeddings into the glove format
# word dim1 dim2 dim3 dim4 ... dimX
lines = []
for i, (word, vector) in tqdm(enumerate(latent_vectors.items())):
    line = [word] + [str(x) for x in vector.tolist()]
    lines.append(' '.join(line))

In [ ]:
with open(f"data/embeddings/trained/{experiment_name}.glove.6B.300d.txt", "w", encoding="utf-8") as fp:
    fp.write("\n".join(lines))